In [ ]:
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from Domain.Nutzenergieanalyse import NEAData

import Bevoelkerung
import Domain.General
import Energiebilanz
import General
import Nutzenergieanalyse
import Hauptwohnsitze
import Processor
import Utils


def create_general_data(g_laender_factory: Domain.General.GLaenderFactory):
    g_sektoren_factory = General.GSektorenDefaultFactory()
    g_bereiche_factory = General.GBereicheSimpleFactory()
    g_energietraeger_klassen_factory = General.GEnergietraegerKlassenSimpleFactory()
    g_energietraeger_factory = General.GEnergietraegerSimpleFactory()
    g_data_factory = General.GDataFactory(g_laender_factory, g_sektoren_factory, g_bereiche_factory,
                                          g_energietraeger_klassen_factory, g_energietraeger_factory)
    g_data = g_data_factory.create()
    return g_data

def create_nea_data(g_data: Domain.General.GData):
    nea_data_pickle_factory = Nutzenergieanalyse.NEADataPickleFactory(pathlib.Path('Data/Serialization'))
    nea_data_factory = Nutzenergieanalyse.NEADataETKlassenFactory(nea_data_pickle_factory)
    nea_data = nea_data_factory.create(g_data).data
    return {land.name: {sektor.name: {bereich.name: nea_data[land][sektor][bereich] for bereich in nea_data[land][sektor].keys()} for sektor in nea_data[land].keys()} for land in nea_data.keys()}

def create_eb_data(g_data: Domain.General.GData):
    eb_data_pickle_factory = Energiebilanz.EBDataPickleFactory(pathlib.Path('Data/Serialization'))
    eb_data_factory = Energiebilanz.EBDataETKlassenFactory(eb_data_pickle_factory)
    eb_data = eb_data_factory.create(g_data).data
    return {land.name: {sektor.name: eb_data[land][sektor] for sektor in eb_data[land].keys()} for land in eb_data.keys()}

def create_b_data(g_data: Domain.General.GData):
    b_data_factory = Bevoelkerung.BDataPickleFactory(pathlib.Path('Data/Serialization'), list(general_data.laender.values()))
    b_data = b_data_factory.create(list(general_data.laender.values())).data
    return {land.name: b_data[land] for land in b_data.keys()}

def create_hws_data(g_data: Domain.General.GData):
    hws_data_factory = Hauptwohnsitze.HWSDataPickleFactory(pathlib.Path('Data/Serialization'), list(general_data.laender.values()))
    hws_data = hws_data_factory.create(list(general_data.laender.values())).data
    return {land.name: hws_data[land] for land in hws_data.keys()}
    
laender_factory = General.GBundeslaenderFactory()
general_data = create_general_data(laender_factory)

nea_data = create_nea_data(general_data)
eb_data = create_eb_data(general_data)
b_data = create_b_data(general_data)
#hws_data = create_hws_data(general_data)


In [ ]:
def create_nea_data_sektor(land: str, sektor: str):
    return sum([nea_data[land][sektor][bereich.name] for bereich in general_data.bereiche.values()])

def create_nea_data_land(land: str):
    return sum([create_nea_data_sektor(land, sektor.name) for sektor in general_data.sektoren.values()])

def create_nea_data_bereich_oesterreich(sektor: str, bereich: str):
    return sum([nea_data[land.name][sektor][bereich] for land in general_data.laender.values()])

def create_nea_data_sektor_oesterreich(sektor: str):
    return sum([create_nea_data_sektor(land.name, sektor) for land in general_data.laender.values()])

def create_nea_data_oesterreich():
    return sum([create_nea_data_land(land.name) for land in general_data.laender.values()])


In [ ]:
def clean_negative_values(data):
    df = data.copy()
    df[df < 0] = 0
    return df

def calc_relative(df: pd.DataFrame):
    return (df.transpose() / df.sum(axis=1)).transpose()


In [ ]:
extrapolation = Utils.DFSARIMAXExtrapolation()
extension = Utils.DFNanExtension()
extrapolation_extension = Utils.DFExtrapolationExtension(extrapolation)


In [ ]:
def extend_nea_data(land: str, sektor: str, bereich: str):
    s = create_nea_data_sektor(land, sektor)
    b = nea_data[land][sektor][bereich]
    rel = (b / s).fillna(0)
    eb = eb_data[land][sektor]
    rel_ex = extrapolation_extension.run(rel, pd.PeriodIndex(pd.period_range(start='1993', end='2040', freq='A'), name=rel.index.name))
    return rel_ex * eb

def extend_all_nea_data():
    return {land: {sektor: {bereich: extend_nea_data(land, sektor, bereich) for bereich in nea_data[land][sektor].keys()} for sektor in nea_data[land].keys()} for land in nea_data.keys()}

#nea_data_ex = extend_all_nea_data()


In [ ]:
extend_nea_data('wien', 'Wohngebäude', 'Heizen')

In [ ]:
nea_data_ex['wien']['Wohngebäude']['Heizen']

In [ ]:
def run_scenario(df: pd.DataFrame, wanted: pd.Series):
    et = wanted.name
    
    df_relative = calc_relative(df)
    df_relative_without_et = df_relative.drop(et, axis=1)
    df_relative_without_et_relative = calc_relative(df_relative_without_et)
    
    to_sub = df[et] - wanted
    to_add = (df_relative_without_et_relative.transpose() * to_sub).transpose()
    to_add[et] = to_sub * -1

    result = df + to_add
    result_reordered = result.reindex(columns=df.columns)
    return result_reordered
    
def run_scenarios(df: pd.DataFrame, wanted: pd.DataFrame):
    global_result = df.copy()
    for et in wanted.columns:
        local_result = run_scenario(global_result, wanted[et])
        global_result = local_result.drop(et, axis=1)
    result = pd.concat([global_result, wanted], axis=1)
    result_reordered = result.reindex(columns=df.columns)
    return result_reordered
    

In [ ]:
def create_linear_scenario(series: pd.Series, start: str, end: str, end_value):
    et = series.name
    index = series[start:end][1:].index
    start_value = series[start]
    decrement = start_value / (len(index))
    start_series = series[:start]
    wanted_series = pd.Series([start_value - decrement * (i + 1) for i in range(len(index))], index=index)
    end_series = series[end:][1:]
    result = pd.concat([start_series, wanted_series, end_series])
    result.name = series.name
    return result


### Generate result

per person

use moving average

interpolate population since some values are missing

In [ ]:
def create_data_bereich(land: str, sektor: str, bereich: str):
    nea = nea_data_ex[land][sektor][bereich]
    bevoelkerung = b_data[land]
    bevoelkerung = pd.Series(bevoelkerung, pd.period_range('1988', '2040', freq='A')).interpolate()
    nea_pp = (nea.transpose() / bevoelkerung['1988':'2020']).transpose().rolling(3).mean()['1990':]
    nea_pp_ex = extrapolation_extension.run(nea_pp, pd.PeriodIndex(pd.period_range(start=nea_pp.index[0], end='2040', freq='A'), name=nea_pp.index.name))
    return (clean_negative_values(nea_pp_ex.transpose()) * bevoelkerung['1990':]).transpose()

def create_data_sektor(land: str, sektor: str):
    return sum([create_data_bereich(land, sektor, bereich.name) for bereich in general_data.bereiche.values()])

def create_data_land(land: str):
    return sum([create_data_sektor(land, sektor.name) for sektor in general_data.sektoren.values()])

def create_data_oesterreich():
    return sum([create_data_land(land.name) for land in general_data.laender.values()])


### Test Szenario:

2040 -> 0 gas

2035 -> 0 oel

2030 -> 0 kohle

na: scheitholz bleibt gleich

na: fernwaerme, strom, umweltwaereme -> 1,2,3% steigung

In [ ]:
def create_test_scenario(df):
    test_scenario_gas = create_linear_scenario(df['Fossile gasförmig'], '2020', '2040', 0)

    test_scenario_oel = create_linear_scenario(df['Fossile flüssig'], '2020', '2035', 0)
    test_scenario_oel = create_linear_scenario(test_scenario_oel, '2035', '2040', 0)

    test_scenario_kohle = create_linear_scenario(df['Fossile fest'], '2020', '2030', 0)
    test_scenario_kohle = create_linear_scenario(test_scenario_kohle, '2030', '2040', 0)

    return pd.concat([test_scenario_gas, test_scenario_oel, test_scenario_kohle], axis=1)

In [ ]:
def since_epoch(period):
        return (period - pd.Period('1970-01-01', freq=period.freq)).n

def plot_result(data, scenario):
    matplotlib.rcParams["figure.figsize"] = (20,10)
    matplotlib.rcParams['axes.labelsize'] = 14
    matplotlib.rcParams['xtick.labelsize'] = 12
    matplotlib.rcParams['ytick.labelsize'] = 12
    matplotlib.rcParams['text.color'] = 'k'

    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True)

    data.plot(ax=ax1)
    ax1.set_title('Data')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('TJ')
    ax1.axvspan(since_epoch(data['2020':].index[0]), since_epoch(data.index[-1:][0]), color='#808080', alpha=0.2)

    scenario.plot(ax=ax2)
    ax2.set_title('Szenario')
    ax2.set_xlabel('Year')
    ax2.set_ylabel('TJ')
    ax2.axvspan(since_epoch(scenario['2020':].index[0]), since_epoch(scenario.index[-1:][0]), color='#808080', alpha=0.2)

    fig.autofmt_xdate()
    plt.tight_layout()

    plt.show()

In [ ]:
heizen_ph_wien = create_data_bereich('oberoesterreich', 'Wohngebäude', 'Heizen')

test_scenario_heizen_ph_wien = create_test_scenario(heizen_ph_wien)
test_scenario_heizen_ph_wien_result = run_scenarios(heizen_ph_wien, test_scenario_heizen_ph_wien)

plot_result(heizen_ph_wien, test_scenario_heizen_ph_wien_result)

In [ ]:
ph_wien = create_data_sektor('wien', 'Wohngebäude')

test_scenario_ph_wien = create_test_scenario(ph_wien)
test_scenario_ph_wien_result = run_scenarios(ph_wien, test_scenario_ph_wien)

plot_result(ph_wien, test_scenario_ph_wien_result)


In [ ]:
wien = create_data_land('wien')

test_scenario_wien = create_test_scenario(wien)
test_scenario_wien_result = run_scenarios(wien, test_scenario_wien)

plot_result(wien, test_scenario_wien_result)


In [ ]:
wien.sum(axis=1).plot()

In [ ]:
oesterreich = create_data_oesterreich()

test_scenario_oesterreich = create_test_scenario(oesterreich)
test_scenario_oesterreich_result = run_scenarios(oesterreich, test_scenario_oesterreich)

plot_result(oesterreich, test_scenario_oesterreich_result)

In [ ]:
oesterreich.sum(axis=1).plot()

In [ ]:
b_data_sum = pd.DataFrame([b_data[land.name] for land in general_data.laender.values()])

matplotlib.rcParams["figure.figsize"] = (20,10)
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

fig, ax1 = plt.subplots(nrows=1, ncols=1, sharex=True)

b_data_sum.transpose().plot(ax=ax1)
ax1.set_title('Population')
ax1.set_xlabel('Year')
ax1.set_ylabel('Millions')

fig.autofmt_xdate()
plt.tight_layout()

plt.show()

#b_data_sum.transpose().plot()

In [ ]:

def create_limit_scenario(series: pd.Series, start: str, factor):
    et = series.name
    
    limit_series = series[start:end][1:]
    index = limit_series.index
    start_value = series[start]
    
    limit = pd.Series([start_value * factor^(i+1) for i in range(len(index))], index=index)
    
    
    decrement = start_value / (len(index))
    start_series = series[:start]
    wanted_series = pd.Series([start_value * factor^(i+1) for i in range(len(index))], index=index)
    end_series = series[end:][1:]
    result = pd.concat([start_series, wanted_series, end_series])
    result.name = series.name
    return result